In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import xlrd
import imageio
from scipy import sqrt, pi, arctan2, cos, sin # used for HoF
from scipy.ndimage import uniform_filter # used for hoF
import time
import random
from sklearn.metrics import confusion_matrix
import sklearn.metrics as skm
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import BatchNormalization
from keras.layers.convolutional import Convolution3D, MaxPooling3D
from keras.optimizers import SGD, RMSprop
from keras.callbacks import ModelCheckpoint,CSVLogger,LearningRateScheduler 
from keras.utils import np_utils, generic_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from keras import backend as K
import sys
from keras.models import Sequential
import tensorflow as tf
import matplotlib.pyplot as plt
import dlib
import re
import collections
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import plot_model

Using TensorFlow backend.


In [4]:
def cv2plt(img):
    plt.axis('off')
    if np.size(img.shape) == 3:
        plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))#convert first BGR (cv2 format)to matplotlib format (RGB)
    else:
        plt.imshow(img,cmap='gray',vmin=0,vmax=255)
        print("NOWHERE")
    plt.show()

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17248002527952040147
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6630704415
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15418498957484888101
physical_device_desc: "device: 0, name: GeForce RTX 2070 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


True

In [6]:
# casmeII RAW selected video frames location
data_loc=r"C:\Users\ajayv\Desktop\Db-project2\CASME-II\CASME2_RAW_selected\CASME2_RAW_selected" # r -> raw string reproduce

# casmeII video frame label location
label_file_path = r'C:\Users\ajayv\Desktop\Db-project2\CASME-II'

#### extracting the frame name from the location... alternate to building frame_name_list

def extract_subjectnumber(string):

# extract the file name from 
    a = string
    b= string[:-2]
    if b in a:
        c= a.replace(b,'')
    return int(c)


def extract_label(sub_num,frame_name):
    #sub num -> is sub folder number in integer.
    #sub num+frame_name represent a unique key in the label file to identify the label
    tobematched = str(sub_num)+frame_name
    for a in df_label_file.index:
        if df_label_file["sub-file"].iloc[a] == tobematched:

            label = df_label_file["Estimated Emotion"].iloc[a]

            break
    return label


df_label_file = pd.read_excel(os.path.join(label_file_path, 'CASME2-coding-20190701.xlsx'))


df_label_file["sub-file"] = (df_label_file["Subject"]).astype(str) + df_label_file["Filename"].astype(str) 




#subject folders (same subject multiple emotion sequences)        
folder_list = os.listdir(data_loc)

frame_location_list=[] #frame absolute location list
frame_folder_list=[] #frame subject folder number 
frame_name_list=[] #frame name (same name may exist in more then one sub folder)
frame_label=[]#frame's label for a given absolute location

for a in folder_list:
    file_folder_loc = os.path.join(data_loc, a)

    files_in_folder = os.listdir(file_folder_loc)

    for b in files_in_folder:
        
        file_loc = os.path.join(file_folder_loc,b)
        frame_location_list.append(file_loc) 
        
        sub_num = extract_subjectnumber(a)
        frame_folder_list.append(sub_num)
        label = extract_label(sub_num,b)
        frame_label.append(label)

    
        frame_name_list.append(b) #save file names for building the labels later with lookup at CASME2-coding-20190701.xlsx


        frame_location_list[0], #sequence absolute locations
        frame_folder_list[0], #subject folder name
        frame_name_list[0], #sequence folder name (same sequence name may exist at different subjects)
        frame_label[0] ) #sequence label

frame loc, frame folder, frame name, frame label C:\Users\ajayv\Desktop\Db-project2\CASME-II\CASME2_RAW_selected\CASME2_RAW_selected\sub01\EP02_01f 1 EP02_01f happiness


In [7]:
# DLib Face Detection
# predictor_path = "shape_predictor_68_face_landmarks.dat"
predictor_path=r"C:\Users\ajayv\iCloudDrive\Documents\01-knowledge\01-NUS-MTechISS\01-2020sem1\project2\code\shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(predictor_path)
detector = dlib.get_frontal_face_detector()

In [8]:
def get_landmark(img):
    
   # cv2plt(img)
    rects = detector(img, 1)
  #  print(rects[0])
    if len(rects) > 1:
        pass
    if len(rects) == 0:
        pass
    ans = np.matrix([[p.x, p.y] for p in predictor(img, rects[0]).parts()])
    return ans

In [9]:

#to sort images: temporal sequence frame processing
numbers=re.compile(r'(\d+)')
def numericalsort(value):
    parts=numbers.split(value)
    parts[1::2]=map(int, parts[1::2])
    return parts

In [10]:
# Reference: https://github.com/colincsl/pyKinectTools/blob/master/pyKinectTools/algs/HistogramOfOpticalFlow.py

def hof(flow, orientations=9, pixels_per_cell=(8, 8),
        cells_per_block=(2, 2), normalise=False, motion_threshold=1.):

    flow = np.atleast_2d(flow)

    if flow.ndim < 3:
        raise ValueError("Requires dense flow in both directions")

    if normalise:
        flow = sqrt(flow)


    if flow.dtype.kind == 'u':
        # convert uint image to float
        # to avoid problems with subtracting unsigned numbers in np.diff()
        flow = flow.astype('float')

    gx = np.zeros(flow.shape[:2])
    gy = np.zeros(flow.shape[:2])
    # gx[:, :-1] = np.diff(flow[:,:,1], n=1, axis=1)
    # gy[:-1, :] = np.diff(flow[:,:,0], n=1, axis=0)

    gx = flow[:,:,1]
    gy = flow[:,:,0]

    magnitude = sqrt(gx**2 + gy**2)
    orientation = arctan2(gy, gx) * (180 / pi) % 180

    sy, sx = flow.shape[:2]
    cx, cy = pixels_per_cell
    bx, by = cells_per_block

    n_cellsx = int(np.floor(sx // cx))  # number of cells in x
    n_cellsy = int(np.floor(sy // cy))  # number of cells in y

    # compute orientations integral images
    orientation_histogram = np.zeros((n_cellsy, n_cellsx, orientations))
    subsample = np.index_exp[int(cy / 2):cy * n_cellsy:cy, int(cx / 2):cx * n_cellsx:cx]
    for i in range(orientations-1):
        #create new integral image for this orientation
        # isolate orientations in this range

        temp_ori = np.where(orientation < 180 / orientations * (i + 1),
                            orientation, -1)
        temp_ori = np.where(orientation >= 180 / orientations * i,
                            temp_ori, -1)
        # select magnitudes for those orientations
        cond2 = (temp_ori > -1) * (magnitude > motion_threshold)
        temp_mag = np.where(cond2, magnitude, 0)

        temp_filt = uniform_filter(temp_mag, size=(cy, cx))
        orientation_histogram[:, :, i] = temp_filt[subsample]


    temp_mag = np.where(magnitude <= motion_threshold, magnitude, 0)

    temp_filt = uniform_filter(temp_mag, size=(cy, cx))
    orientation_histogram[:, :, -1] = temp_filt[subsample]


    n_blocksx = (n_cellsx - bx) + 1
    n_blocksy = (n_cellsy - by) + 1
    normalised_blocks = np.zeros((n_blocksy, n_blocksx,
                                  by, bx, orientations))

    for x in range(n_blocksx):
        for y in range(n_blocksy):
            block = orientation_histogram[y:y+by, x:x+bx, :]
            eps = 1e-5
            normalised_blocks[y, x, :] = block / sqrt(block.sum()**2 + eps)

    return normalised_blocks.ravel()

In [11]:
#extract Histogram of Optical Flow features for a given consequetive frames. Replace frame by extracted HOG
def extract_hof_feature(seq_location_list,seq_label):
    feature_hof = []
    label_list = []
    img_width = 128
    img_height = 64
    
    print("total number of sequences are:", len(seq_location_list))
    
    #each pass processes a sequence of frame in 
    for a in range(0,len(seq_location_list)): #idx is to display progress, a is to iterate through CASMEII/SMIC db
       # Display the progress
        if ((a % 10) == 0):
            print("process sequence %d /%d" % (a, len(seq_location_list)))
        
        hof_feature_all = []

        
        filepath = seq_location_list[a]
        
        seq_folder = os.listdir(filepath) # contains the frames for that micro-expression sequence
        
        #double check: sort the folder to get temporal seq 
        sort_seq_folder = sorted(seq_folder, key=numericalsort)
        
        label = seq_label[a]
        
        
        #iterate the frames inside a given sequence
        for b in range(0,len(sort_seq_folder)-1):
            framepath = os.path.join(filepath, sort_seq_folder[b])
            
            if (cv2.imread(framepath) is not None):
                frame = cv2.imread(framepath)
            
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            previousGray = gray
            
            framepath_next = os.path.join(filepath, sort_seq_folder[b+1])
            
            frame_next = cv2.imread(framepath_next)
            
            gray = cv2.cvtColor(frame_next, cv2.COLOR_BGR2GRAY)
            flow = cv2.calcOpticalFlowFarneback(previousGray, gray, 
                                                flow=None, pyr_scale=0.5, levels=5, winsize=11, 
                                                iterations=10, poly_n=5, poly_sigma=1.1, flags=0)
            
            hof_feature_one = hof(flow, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2))
            
            if (len(hof_feature_all) == 0):
                hof_feature_all = hof_feature_one
            else:
                hof_feature_all = np.vstack((hof_feature_all, hof_feature_one))
        
                
        if (len(hof_feature_all) != 0):
            hof_feature_mean = np.mean(hof_feature_all, axis=0)
            feature_hof.append(hof_feature_mean)
            label_list.append(label)      

    return np.array(feature_hof), np.array(label_list)


In [12]:
def get_eye_mouth_img(image):
    
    landmarks = get_landmark(image)
    numpylandmarks = np.asarray(landmarks)
    eye_image = image[numpylandmarks[19][1]:numpylandmarks[1][1], numpylandmarks[1][0]:numpylandmarks[15][0]]
    eye_image = cv2.resize(eye_image, (32, 32), interpolation = cv2.INTER_AREA)
    eye_image = cv2.cvtColor(eye_image, cv2.COLOR_BGR2GRAY)
    nose_mouth_image = image[numpylandmarks[2][1]:numpylandmarks[6][1], numpylandmarks[2][0]:numpylandmarks[14][0]]
    nose_mouth_image = cv2.resize(nose_mouth_image, (32, 32), interpolation = cv2.INTER_AREA)
    nose_mouth_image = cv2.cvtColor(nose_mouth_image, cv2.COLOR_BGR2GRAY)
    
    return eye_image, nose_mouth_image

In [13]:
#extract Histogram of Optical Flow features for eye mouth for a given consequetive frames. Replace frame by extracted HOG
def extract_hof_eye_mouth_feature(seq_location_list,seq_label):
    feature_hof = []
    label_list = []
    img_width = 128
    img_height = 64
    
    print("total number of sequences are:", len(seq_location_list))
    
    #each pass processes a sequence of frame in 
    for a in range(0,len(seq_location_list)): #idx is to display progress, a is to iterate through CASMEII/SMIC db
       # Display the progress
    
    
        print("folder time", time.time())
        
        if ((a % 10) == 0):
            print("process sequence %d /%d" % (a, len(seq_location_list)))
        
        hof_eye_feature_all = [] # eye features
        hof_nose_mouth_feature_all = [] # eye features

        
        filepath = seq_location_list[a]
        label = seq_label[a]
        
        seq_folder = os.listdir(filepath) # contains the frames for that micro-expression sequence
        sort_seq_folder = sorted(seq_folder, key=numericalsort)
        
        for b in range(0,len(sort_seq_folder)-1):
            framepath = os.path.join(filepath, sort_seq_folder[b])
            
            

            if (cv2.imread(framepath) is not None):
                frame = cv2.imread(framepath)
            
            framepath_next = os.path.join(filepath, sort_seq_folder[b+1])
            
            frame_next = cv2.imread(framepath_next)
            
            
            previous_eye, previous_nose_mouth = get_eye_mouth_img(frame)
            
            eye, nose_mouth = get_eye_mouth_img(frame_next)
            
            
            flow_eye = cv2.calcOpticalFlowFarneback(previous_eye, eye, 
                                                flow=None, pyr_scale=0.5, levels=5, winsize=11, 
                                                iterations=10, poly_n=5, poly_sigma=1.1, flags=0)
        
            flow_nose_mouth = cv2.calcOpticalFlowFarneback(previous_nose_mouth, nose_mouth, 
                                                flow=None, pyr_scale=0.5, levels=5, winsize=11, 
                                                iterations=10, poly_n=5, poly_sigma=1.1, flags=0)
        
            
            hof_eye_feature_one = hof(flow_eye, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2))
            hof_nose_mouth_feature_one = hof(flow_nose_mouth, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2))
            
            if (len(hof_eye_feature_all) == 0):
                hof_eye_feature_all = hof_eye_feature_one
                hof_nose_mouth_feature_all = hof_nose_mouth_feature_one
                
                
            else:
                hof_eye_feature_all = np.vstack((hof_eye_feature_all, hof_eye_feature_one))
                hof_nose_mouth_feature_all = np.vstack((hof_nose_mouth_feature_all, hof_nose_mouth_feature_one))
        
                
        if (len(hof_eye_feature_all) != 0):
            hof_eye_feature_mean = np.mean(hof_eye_feature_all, axis=0)
            hof_nose_mouth_feature_mean = np.mean(hof_nose_mouth_feature_all, axis=0)
            feature_hof.append(np.vstack((hof_eye_feature_mean,hof_nose_mouth_feature_mean)))
            label_list.append(label)      

    return np.array(feature_hof), np.array(label_list)

In [14]:
def train_test_set(features,labels):
    
    test_samples = []
    
    train_features = []
    test_features = []
    train_labels = []
    test_labels = []
    
    num_samples = len(features)
    
    num_test = int(0.2*num_samples)
    
    random.seed(0)
    
    test_list_index = random.sample(range(0, num_samples-1), num_test)
    
    print(test_list_index)
    
    for a in test_list_index:
        test_labels.append(labels[a])
        test_features.append(features[a])
        
    train_features = [i for j, i in enumerate(features) if j not in test_list_index]
    train_labels = [i for j, i in enumerate(labels) if j not in test_list_index]
    
    return np.array(train_features), np.array(train_labels), np.array(test_features), np.array(test_labels)


In [ ]:
#extract features at face

now = time.time()

feature_hof, label_hof = extract_hof_feature(frame_location_list,frame_label)      

print("total extraction time", time.time()-now)

In [ ]:
np.save('.\hof_feature_eye_nose, feature_hof',flatt_feature_hof)
np.save('.\label_hof_eye_nose, feature_hof',label_hof)